This is a vectorized version of the environment. How does it work? Simply put, instead of running a single environment, we're running them into batches (vectorization). At each time step, instead of formulating a single action $a$, we'll define it as a vector $a=[0, .., n_\text{envs}]$ where each entry corresponds to the action to be performed for an environment.

Thus, if you run 4 simulatenous environments, your observations space becomes (4, num_observations). Because we use a step size of 10 and have a total of 9 snesors, each observation will result into a space of (40,9) elements. The core idea is to speed up the inference and training time of the model instead of querying a single environment.

In our implementation, based on what model you decide, you should take as inputs the observations reshaped to your liking, and predict the actions $a$ from your policy.

In [1]:
import numpy as np
from student_client.student_gym_env_vectorized import create_student_gym_env_vectorized

env = create_student_gym_env_vectorized(
            num_envs=4,
            step_size=10,
            user_token='vec1'
        )

2026-02-11 18:12:30,008 - httpx - INFO - HTTP Request: POST http://rlchallenge.orailix.com/api/v1/session/create "HTTP/1.1 200 OK"
2026-02-11 18:12:30,009 - student_client.student_gym_env_vectorized - INFO - Created new session: 84727f83-4ac7-4909-9832-2316cd8a8079
2026-02-11 18:12:31,597 - httpx - INFO - HTTP Request: POST http://rlchallenge.orailix.com/api/v1/episode/create "HTTP/1.1 200 OK"
2026-02-11 18:12:31,599 - student_client.student_gym_env_vectorized - INFO - Created new episode 1/4: efbdf6a3-e894-4e19-b3b0-400ef10ddd5e
2026-02-11 18:12:32,593 - httpx - INFO - HTTP Request: POST http://rlchallenge.orailix.com/api/v1/episode/create "HTTP/1.1 200 OK"
2026-02-11 18:12:32,594 - student_client.student_gym_env_vectorized - INFO - Created new episode 2/4: fa8c96f7-406e-4581-a8fc-815ba68ce0fc
2026-02-11 18:12:33,513 - httpx - INFO - HTTP Request: POST http://rlchallenge.orailix.com/api/v1/episode/create "HTTP/1.1 200 OK"
2026-02-11 18:12:33,514 - student_client.student_gym_env_vector

In [2]:
print(f"Environment created with {env.num_envs} parallel environments")
print(f"   Episode IDs: {env.episode_ids}")

# Reset all environments
print(f"\n🔄 Resetting all environments...")
observations, infos = env.reset()

print(f"   Observations shape: {observations.shape}")
print(f"   First observation: {observations[0]}")

Environment created with 4 parallel environments
   Episode IDs: ['efbdf6a3-e894-4e19-b3b0-400ef10ddd5e', 'fa8c96f7-406e-4581-a8fc-815ba68ce0fc', '06938f2d-9233-454f-90ac-890b327444e7', '2ef8afb9-a837-4c6d-a7f1-3c01df64dab7']

🔄 Resetting all environments...


2026-02-11 18:12:37,858 - httpx - INFO - HTTP Request: POST http://rlchallenge.orailix.com/api/v1/episode/vectorized_reset "HTTP/1.1 200 OK"
2026-02-11 18:12:37,860 - student_client.student_gym_env_vectorized - INFO - All 4 environments reset successfully


   Observations shape: (4, 9)
   First observation: [7.9825012e+02 1.9426363e+04 3.3622369e+02 1.1220037e+03 3.7247667e-01
 1.3683762e+06 3.9601526e+03 0.0000000e+00 1.0180057e+01]


## Training / Iterations

Here you can iterate through the vectorized environments. You'll notice that actions are a vector where each entry corresponds to the associated environment in the vector.

In A), we automatically reset the envs that have terminated so you can continue for an indefinite amount of steps. As environments don't have the same length, they stop at different times, this helps you reset terminated episodes on the fly.

Tips:
- The step_size return many observations, should you feed each one-by-one in your model, or the full step_size=10 one? The choice is yours!
- There exists multiple ways of exploring the dataset

In [3]:
for step in range(40):

    # A) Check if any environments terminated
    terminated_envs = env.get_terminated_env_indices()
    if terminated_envs:
        print(f"   ⚠️  Environments {terminated_envs} terminated")
        reset_obs, reset_infos = env.reset_specific_envs(terminated_envs)
        for i, env_id in enumerate(terminated_envs):
            infos[env_id] = reset_infos[i] # reset previous info dict

    # Generate random actions for all environments
    actions = np.random.randint(0, 3, size=env.num_envs)

    print(f"\n   Step {step + 1}:")
    print(f"      Actions: {actions}")

    # Take step
    observations, rewards, terminateds, truncateds, infos = env.step(actions)

    print(f"      Rewards: {rewards}")
    print(f"      Terminated: {terminateds}")
    print(f"      Active environments: {env.get_active_count()}/{env.num_envs}")

    # Show filtered info (production mode)
    for i, info in enumerate(infos):
        print(f"      Env {i} info: {info}")

env.close()


   Step 1:
      Actions: [2 1 0 1]


2026-02-11 18:12:41,668 - httpx - INFO - HTTP Request: POST http://rlchallenge.orailix.com/api/v1/episode/vectorized_step "HTTP/1.1 200 OK"


      Rewards: [ 138.9658  -519.30054  464.9032  -518.9286 ]
      Terminated: [ True False False False]
      Active environments: 3/4
      Env 0 info: {'step': 1, 'terminated': True, 'truncated': False}
      Env 1 info: {'step': 9, 'terminated': False, 'truncated': False}
      Env 2 info: {'step': 9, 'terminated': False, 'truncated': False}
      Env 3 info: {'step': 9, 'terminated': False, 'truncated': False}
   ⚠️  Environments [0] terminated


2026-02-11 18:12:42,542 - httpx - INFO - HTTP Request: POST http://rlchallenge.orailix.com/api/v1/episode/vectorized_reset "HTTP/1.1 200 OK"



   Step 2:
      Actions: [1 1 2 0]


KeyboardInterrupt: 

In [4]:
observations

[array([[7.9825012e+02, 1.9426363e+04, 3.3622369e+02, 1.1220037e+03,
         3.7247667e-01, 1.3683762e+06, 3.9601526e+03, 0.0000000e+00,
         1.0180057e+01]], dtype=float32),
 array([[7.9477319e+02, 1.9343662e+04, 3.3467184e+02, 1.1164084e+03,
         3.7129980e-01, 1.3708640e+06, 3.9513721e+03, 0.0000000e+00,
         9.1000004e+00],
        [7.9491779e+02, 1.9378010e+04, 3.3464154e+02, 1.1168699e+03,
         3.7137550e-01, 1.3694258e+06, 3.9509656e+03, 0.0000000e+00,
         9.1000004e+00],
        [7.9479761e+02, 1.9360307e+04, 3.3461899e+02, 1.1165081e+03,
         3.7131017e-01, 1.3703155e+06, 3.9509536e+03, 0.0000000e+00,
         9.1000004e+00],
        [7.9617084e+02, 1.9366996e+04, 3.3458795e+02, 1.1160082e+03,
         3.7123212e-01, 1.3777810e+06, 3.9509363e+03, 0.0000000e+00,
         9.1504993e+00],
        [7.9502557e+02, 1.9339953e+04, 3.3461966e+02, 1.1160654e+03,
         3.7123680e-01, 1.3730731e+06, 3.9511396e+03, 0.0000000e+00,
         9.1000004e+00],
     